In [1]:
!pip install biosppy
!pip install antropy

     |████████████████████████████████| 85 kB 2.3 MB/s 
  Created wheel for biosppy: filename=biosppy-0.7.3-py2.py3-none-any.whl size=95430 sha256=c0739fbd9ec4c78285c23d470947a9f7d1885fc71f2c3419b9e4a2029edee49c
  Stored in directory: /root/.cache/pip/wheels/2f/4f/8f/28b2adc462d7e37245507324f4817ce1c64ef2464f099f4f0b
Successfully built biosppy
     |████████████████████████████████| 49 kB 2.4 MB/s 
  Created wheel for antropy: filename=antropy-0.1.4-py3-none-any.whl size=16158 sha256=e06c0aff96380dc9f2b3b6a10d9b8c2df8d56da0e718c078dca86879b0998fdf
  Stored in directory: /root/.cache/pip/wheels/d4/cc/6b/28fa7036bdabc6f1f92fd6a108ae259dbb48f107f676d6b491
Successfully built antropy


In [2]:
import biosppy.signals.ecg as ecg
import antropy as ant
import matplotlib.pyplot as plt

In [3]:
import torch
import numpy as np
import pandas as pd

train_X = pd.read_csv('X_train.csv')
train_X_tensor = torch.tensor(train_X.values)[:,1:]
train_y = pd.read_csv("y_train.csv")
train_y_tensor = torch.tensor(train_y.values)[:,1:]

test_X = pd.read_csv('X_test.csv')
test_X_tensor = torch.tensor(test_X.values)[:,1:]

In [4]:
test_X_tensor.shape

torch.Size([3411, 17842])

In [5]:
X_train = train_X_tensor.numpy()
y_train = train_y_tensor.numpy()

In [6]:
y_train.shape

(5117, 1)

In [8]:
X_train.shape

(5117, 17842)

In [7]:
y_train = np.squeeze(y_train, axis=1)

In [ ]:
#@title hide pls
signal = X_train[1]
signal = signal[~np.isnan(signal)] #drop nans
r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
# print(r_peaks)

if len(r_peaks) >= 2:
    #print(ecg.extract_heartbeats(signal, r_peaks, 300))
    beats = ecg.extract_heartbeats(signal, r_peaks, 300)['templates']
beats.shape
r_peaks

In [52]:
!pip install pyhrv
from pyhrv.frequency_domain import welch_psd

     |████████████████████████████████| 3.2 MB 4.3 MB/s 
     |████████████████████████████████| 228 kB 51.8 MB/s 
  Created wheel for spectrum: filename=spectrum-0.8.0-cp37-cp37m-linux_x86_64.whl size=234833 sha256=c79a54793d1d9be7a2f38b8a9bb68a6501823db291d28c9cce53332c69121868
  Stored in directory: /root/.cache/pip/wheels/ac/a0/2b/aba9ac3cce046bc454a44d0db5b711b4e927a0dd33a93e9d98
Successfully built spectrum


In [93]:
fbands = {'ulf': (0.0, 0.033), 'vlf': (0.033, 0.04), 'lf': (0.04, 0.15), 'hf': (0.15, 0.4)} #WHY CANT I ADD 5th frequency?
# fbands = {'vlf': (0.033, 0.04), 'lf': (0.04, 0.15), 'hf': (0.15, 0.4),"noise":(0.4,100)} #WHY CANT I ADD 5th frequency?

In [ ]:
#@title hide pls
signal = X_train[0]
signal = signal[~np.isnan(signal)] #drop nans

res = ecg.ecg(signal, sampling_rate=300, show=False)
r_peaks = res['rpeaks']
heart_rates = res['heart_rate']

# else:
#   print(f"len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
# r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
# extracted_rpeaks.append([signal,r_peaks])
print(heart_rates)
print(r_peaks.shape)

In [90]:
extracted_rpeaks = []

In [91]:
for i in range(len(X_train)):
    signal = X_train[i]
    signal = signal[~np.isnan(signal)] #drop nans
    if (i%500==0):
      print(i)
    res = ecg.ecg(signal, sampling_rate=300, show=False)
    r_peaks = res['rpeaks']
    heart_rates = res["heart_rate"]
    if len(r_peaks) < 5:
      print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # else:
    #   print(f"len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
    extracted_rpeaks.append([signal,r_peaks,heart_rates])

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [92]:
len(extracted_rpeaks)

5117

In [ ]:
count =0
diffs = []
for i in range(len(y_train[y_train==3])):
  signal = X_train[y_train==3][i]
  label = y_train[y_train==3][i]
  signal = signal[~np.isnan(signal)] #drop nans

  res = ecg.ecg(signal, sampling_rate=300, show=False, )
  r_peaks = res['rpeaks']
  heart_rates = res["heart_rate"]
  beats = res["templates"]
  filtered = res["filtered"]
  r_peaks_s = r_peaks/300
  freqs = 20*np.log10(np.abs(np.fft.rfft(signal)))
  f = np.linspace(0, 300/2, len(freqs))
  plt.plot(f, freqs)
  # a = welch_psd(rpeaks=r_peaks_s, fbands = fbands, show=False, show_param=False, legend=False)#, fbands=None)
  # res_filt = ecg.ecg(filtered, sampling_rate=300, show=False, )
  # beats_filt = res_filt["templates"]
  # print(np.mean(np.mean(beats,axis=0)-np.mean(beats_filt,axis=0)))

  #print(f"num of rpeaks if {len(r_peaks)} and avg heartrate is {np.mean(heart_rates)}")
  #print(f"I have {len(r_peaks)} pulses but I should have {np.mean(heart_rates[2:])*len(signal)/18000}")
  # c = np.mean(sorted(heart_rates[:-2]))*len(signal)/18000
  # diff = np.abs((len(r_peaks)/c)-1)
  # print(diff)
  # diffs.append(diff)
# print(np.mean(np.array(diffs)))

  # print(r_peaks)
  # print(r_peaks.shape)
  # print(heart_rates[:15])
  # print(heart_rates.shape)
  # print(beats)
  # print(beats.shape)

In [154]:
import scipy.signal

In [162]:
signal = X_train[y_train==0][8]
label = y_train[y_train==3][0]
signal = signal[~np.isnan(signal)] #drop nans
f, Pxx_spec = scipy.signal.welch(signal, 300, 'flattop', 1024, scaling='spectrum')
Pxx_spec.shape

(513,)

In [165]:
Pxx_spec[0::10].shape

(52,)

In [153]:
signal = X_train[y_train==0][6]
label = y_train[y_train==3][0]
signal = signal[~np.isnan(signal)] #drop nans
res = ecg.ecg(signal, sampling_rate=300, show=False, )
r_peaks = res['rpeaks']
heart_rates = res["heart_rate"]
beats = res["templates"]
filtered = res["filtered"]
r_peaks_s = r_peaks/300
freqs = 20*np.log10(np.abs(np.fft.rfft(signal)))
f = np.linspace(0, 300/2, len(freqs))
len(freqs)

4122

In [76]:
len(freqs)

8406

In [166]:
extracted_features_l = np.ones((5117,5*27+5*4+2+2+2+2*4+1+1+61))

In [ ]:
for i in range(len(extracted_rpeaks)):
    signal, r_peaks, heart_rates = (extracted_rpeaks[i][0],extracted_rpeaks[i][1],extracted_rpeaks[i][2])
    if (i%500==0):
      print(f"processing point {i}")
    if len(r_peaks) >= 5:
        #print(ecg.extract_heartbeats(signal, r_peaks, 300))
        beats = ecg.extract_heartbeats(signal, r_peaks, 300,before=0.45,after=0.45)['templates']
        if len(beats) != 0:
            r_diffs = np.diff(r_peaks)
            mu = np.mean(beats, axis=0)[0::10] 
            mini = np.amin(beats, axis=0)[0::10] 
            maxa = np.amax(beats, axis=0)[0::10]  
            var = np.std(beats, axis=0)[0::10] 
            med = np.median(beats, axis=0)[0::10] 
            r_mu = np.mean(r_diffs)
            r_mini = np.amin(r_diffs) 
            r_maxa = np.amax(r_diffs)  
            r_var = np.std(r_diffs)
            r_med = np.median(r_diffs)
            smentropy = ant.sample_entropy(mu)
            appentropy = ant.app_entropy(mu)

            r_diffs_diffs = np.diff(r_diffs)
            r_diffs_mu = np.mean(r_diffs_diffs)
            r_diffs_maxa = np.amax(r_diffs_diffs)
            r_diffs_mini = np.amin(r_diffs_diffs)
            r_diffs_var = np.std(r_diffs_diffs)
            r_diffs_med = np.median(r_diffs_diffs)
            r_diffs_25 = np.percentile(r_diffs_diffs,25)
            r_diffs_75 = np.percentile(r_diffs_diffs,75)
            nn_50 = len(r_diffs_diffs[r_diffs_diffs>15]) / len(r_diffs_diffs)
            nn_20 = len(r_diffs_diffs[r_diffs_diffs>6]) / len(r_diffs_diffs)
            c = np.mean(sorted(heart_rates[:-2]))*len(signal)/18000
            rat = np.abs((len(r_peaks)/c)-1)

            r_diffs_diffs_2 = np.diff(r_diffs_diffs)
            r_diffs2_mu = np.mean(r_diffs_diffs_2)
            r_diffs2_maxa = np.amax(r_diffs_diffs_2)
            r_diffs2_mini = np.amin(r_diffs_diffs_2)
            r_diffs2_var = np.std(r_diffs_diffs_2)
            r_diffs2_med = np.median(r_diffs_diffs_2)
            r_diffs2 = np.array([r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med])

            abla, lospowers = scipy.signal.welch(signal, 300, 'flattop', 1024, scaling='spectrum')
            #heart rate stats
            if len(heart_rates)>0:
              hr_mu = np.mean(heart_rates)
              hr_maxa = np.amax(heart_rates)
              hr_mini = np.amin(heart_rates)
              hr_var = np.std(heart_rates)
              hr_med = np.median(heart_rates)
            else:
              hr_mu = np.nan
              hr_maxa = np.nan
              hr_mini = np.nan
              hr_var = np.nan
              hr_med = np.nan
            r_stats = np.array([r_mu,r_var,r_mini,r_maxa,r_med,r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med, r_diffs_25, r_diffs_75,hr_mu,hr_maxa,hr_mini,hr_var,hr_med,nn_50,nn_20,smentropy,appentropy,rat])
            r_stats = np.concatenate((r_stats,r_diffs2,lospowers[0:10],lospowers[10::10]))
            #HRV STATS

            r_peaks_s = r_peaks/300
            a = welch_psd(rpeaks=r_peaks_s, fbands = fbands, show=False, show_param=False, legend=False)#, fbands=None)
            hrv_stats = np.concatenate((a["fft_peak"],a["fft_abs"],np.array([a["fft_ratio"]])))
            extracted_features_l[i] = np.array(np.concatenate((mu,var,mini,maxa,med,r_stats,hrv_stats)))
    else:  
        print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
        extracted_features_l[i]=np.ones(5*27+5*4+2+2+2+2*4+1+1+61) #np.array([np.ones(18),np.ones(18),np.ones(18),np.ones(18)])

In [ ]:
#@title My title
# for i in range(len(extracted_rpeaks)):

#     signal = signal[~np.isnan(signal)] #drop nans
#     r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
#     # print(r_peaks)

#     if len(r_peaks) >= 5:
#         #print(ecg.extract_heartbeats(signal, r_peaks, 300))
#         beats = ecg.extract_heartbeats(signal, r_peaks, 300)['templates']
#         if len(beats) != 0:
#             r_diffs = np.diff(r_peaks)
#             mu = np.mean(beats, axis=0)[0::10] 
#             mini = np.amin(beats, axis=0)[0::10] 
#             maxa = np.amax(beats, axis=0)[0::10]  
#             var = np.std(beats, axis=0)[0::10] 
#             r_mean = np.mean(r_diffs)
#             r_mini = np.amin(r_diffs) 
#             r_maxa = np.amax(r_diffs)  
#             r_var = np.std(r_peaks)
#             r_med = np.median(r_peaks)
#             # md = np.median(beats, axis=0)
#             r_stats = np.array([r_mean,r_mini,r_maxa,r_var,r_med])

#             #HRV STATS
#             r_peaks_s = r_peaks/300
#             a = welch_psd(rpeaks=r_peaks_s, show=False, show_param=False, legend=False)#, fbands=None)
#             hrv_stats = np.concatenate((a["fft_peak"],a["fft_rel"],a["fft_abs"],a["fft_log"]))
#             return np.array(np.concatenate((mu,var,mini,maxa,r_stats,hrv_stats)))
#         print(f"Inappropriate input encountered: len(r_peaks) was >=5 but len(beats)=0 => peaks = {len(r_peaks)}, len(beats)={len(beats)}, signal length ={len(signal)}")
#     print(f"Inappropriate input encountered: len(r_peaks) was <5 => peaks = {len(r_peaks)}, signal length ={len(signal)}")
#     return np.ones(4*18+5+4*3) #np.array([np.ones(18),np.ones(18),np.ones(18),np.ones(18)])

In [131]:
# extracted_features = np.apply_along_axis(extract_features, 1, X_train)

In [168]:
bool_extracted = (extracted_features_l == np.array(np.ones(5*27+5*4+2+2+2+2*4+1+1+61))).all(axis=1)

In [169]:
bool_extracted

array([False, False, False, ..., False, False, False])

In [170]:
extracted_features_valid = extracted_features_l[~bool_extracted]

In [171]:
extracted_features_valid.shape

(5117, 232)

In [172]:
extracted_features_collapsed = extracted_features_valid#np.concatenate(np.stack(extracted_features_valid,axis=1),axis=1)
labels_collapsed = y_train[~bool_extracted]

In [173]:
extracted_features_collapsed.shape

(5117, 232)

In [131]:
# #check if all inappropriate ones are noise:
# hopefully_noise_3 = y_train[bool_extracted]
# hopefully_noise_3

------------------------------------------------------Classification------------------------------------------------

In [174]:
#Training-val split
extracted_features_collapsed_a = extracted_features_collapsed[:4660]
labels_collapsed_a = labels_collapsed[:4660]
extracted_features_collapsed_b = extracted_features_collapsed[4660:]
labels_collapsed_b = labels_collapsed[4660:]

In [106]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_sample_weight

In [175]:
#Weigh examples
weighted_labels = compute_sample_weight( class_weight= "balanced" , y =labels_collapsed)
weighted_labels 

array([0.42219472, 0.42219472, 0.42219472, ..., 0.42219472, 0.42219472,
       0.86787653])

In [ ]:
#@title hide pls
# !pip install pytorch-tabnet
# from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
# tab = TabNetClassifier(n_d=24,n_a=24)  #TabNetRegressor()
# tab.fit(extracted_features_collapsed_a[:4300], labels_collapsed_a[:4300],eval_set=[(extracted_features_collapsed_a[4300:], labels_collapsed_a[4300:])],patience=150, max_epochs=1000)
# np.sum(labels_collapsed_b == tab.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)
# #got 0.723, and it even didnt train on the last 300 datapoints

In [140]:
# weighted_labels[labels_collapsed==0] *=0.8
# weighted_labels[labels_collapsed==1] *=1.2
# weighted_labels[labels_collapsed==2] *=1.2
# weighted_labels[labels_collapsed==3] *=1.5
# weighted_labels[labels_collapsed==0] *=0.7
# weighted_labels[labels_collapsed==1] *=1.4
# weighted_labels[labels_collapsed==2] *=1.4
# weighted_labels[labels_collapsed==3] *=1.7

In [108]:
!pip install lightgbm

In [109]:
from lightgbm import LGBMClassifier

In [189]:
lgb = LGBMClassifier(n_estimators=2000, min_child_weight=3, learning_rate=0.1, max_depth=9, random_state=0)
lgb.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4660])
np.sum(labels_collapsed_b == lgb.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.8402625820568927

In [143]:
457*0.838

382.966

In [ ]:
pred = lgb.predict(extracted_features_collapsed_b)
tot = 0
count0=0
count1=0
count2=0
count3=0

for i in range(5117-4660):
  signal, r_peaks, heart_rates = (extracted_rpeaks[i+4660][0],extracted_rpeaks[i+4660][1],extracted_rpeaks[i+4660][2])
  if pred[i]!=labels_collapsed_b[i]:
    print(f"actual label is {labels_collapsed_b[i]} predicted is {pred[i]}")
    if labels_collapsed_b[i]==0:
      count0+=1
    if labels_collapsed_b[i]==1:
      count1+=1
    if labels_collapsed_b[i]==2:
      count2+=1
    if labels_collapsed_b[i]==3:
      count3+=1
# i=0
# if labels_collapsed_b[i]==0:
# signal, r_peaks, heart_rates = (extracted_rpeaks[i+4660][0],extracted_rpeaks[i+4660][1],extracted_rpeaks[i+4660][2])
# if pred[i]!=labels_collapsed_b[i]:
#   print(f"actual label is {labels_collapsed_b[i]} predicted is {pred[i]}")
#   plt.plot

In [177]:
lgb = LGBMClassifier(n_estimators=2000, min_child_weight=3, learning_rate=0.1, max_depth=9, random_state=0)
lgb.fit(extracted_features_collapsed, labels_collapsed, weighted_labels)
np.sum(labels_collapsed == lgb.predict(extracted_features_collapsed))/len(labels_collapsed)

1.0

In [49]:
!pip install xgboost
from xgboost import XGBClassifier

In [199]:
xgb = XGBClassifier(n_estimators=1250, min_child_weight=3, learning_rate=0.145, max_depth=9, random_state=0)
xgb.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4660])
np.sum(labels_collapsed_b == xgb.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.838074398249453

In [ ]:
xgb = XGBClassifier(n_estimators=1250, min_child_weight=3, learning_rate=0.1, max_depth=9, random_state=0)
xgb.fit(extracted_features_collapsed, labels_collapsed, weighted_labels)
np.sum(labels_collapsed == xgb.predict(extracted_features_collapsed))/len(labels_collapsed)

In [ ]:
np.sum(labels_collapsed_a == xgb.predict(extracted_features_collapsed_a))/len(labels_collapsed_a)

1.0

2-step process----------------------------------------

In [225]:
new_dps = extracted_features_collapsed
new_labs = np.ones(5117)

In [226]:
new_labs[~(labels_collapsed==3)] = 0
new_labs[labels_collapsed==3] = 1

In [227]:
new_dps_a = new_dps[:4660]
new_labs_a = new_labs[:4660]
new_dps_b = new_dps[4660:]
new_labs_b = new_labs[4660:]

In [232]:
weighted_labels_noise = compute_sample_weight(class_weight= "balanced" , y =new_labs)
np.unique(weighted_labels_noise)

array([ 0.51718213, 15.05      ])

In [233]:
lgb_denoise = LGBMClassifier(n_estimators=1500, learning_rate=0.1, min_child_weight=3, max_depth=9, random_state=0)
lgb_denoise.fit(new_dps_a, new_labs_a, weighted_labels_noise[:4660])
np.sum(new_labs_b== lgb_denoise.predict(new_dps_b))/len(new_labs_b)

0.9693654266958425

In [234]:
new_dps_nono = extracted_features_collapsed[labels_collapsed!=3]
new_labs_nono = labels_collapsed[labels_collapsed!=3]

In [241]:
new_dps_nono_a = new_dps_nono[:4500]
new_dps_nono_b = new_dps_nono[4500:]
new_labs_nono_a = new_labs_nono[:4500]
new_labs_nono_b = new_labs_nono[4500:]

In [242]:
weighted_labels_nono = compute_sample_weight(class_weight= "balanced" , y =new_labs_nono)
np.unique(weighted_labels_nono)

array([0.54422442, 1.11872456, 3.72234763])

In [243]:
lgb_proper = LGBMClassifier(n_estimators=1500, learning_rate=0.1, min_child_weight=3, max_depth=9, random_state=0)
lgb_proper.fit(new_dps_nono_a, new_labs_nono_a, weighted_labels_nono[:4500])
np.sum(new_labs_nono_b== lgb_proper.predict(new_dps_nono_b))/len(new_labs_nono_b)

0.8523489932885906

-----------------------------------OTHER METHODS---------------------------------

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
mlp = make_pipeline(StandardScaler(),MLPClassifier(hidden_layer_sizes=(256, 128, 64), max_iter=1000, alpha=2, verbose=True, solver="adam", learning_rate_init= 0.0005, learning_rate="adaptive", batch_size=512, n_iter_no_change=300))
mlp.fit(extracted_features_collapsed_a[:,:-1], labels_collapsed_a)
np.sum(labels_collapsed_b == mlp.predict(extracted_features_collapsed_b[:,:-1]))/len(labels_collapsed_b)
#gives 0.781

In [ ]:
mlp = make_pipeline(StandardScaler(),MLPClassifier(hidden_layer_sizes=(512, 256, 128), max_iter=1000, alpha=1, verbose=True, solver="adam", learning_rate_init= 0.0005, learning_rate="adaptive", batch_size=512, n_iter_no_change=300))
mlp.fit(extracted_features_collapsed_e, labels_collapsed)
np.sum(labels_collapsed == mlp.predict(extracted_features_collapsed_e))/len(labels_collapsed)
#gives 0.738 for alpha=1, max_iters=4000

In [ ]:
np.sum(labels_collapsed_a == mlp.predict(extracted_features_collapsed_a))/len(labels_collapsed_a)

1.0

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = make_pipeline(StandardScaler(),KNeighborsClassifier(n_neighbors=25, weights='uniform', p=2))
neigh.fit(extracted_features_collapsed_a, labels_collapsed_a)
np.sum(labels_collapsed_b == neigh.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

In [ ]:
np.isinf(extracted_features_collapsed_a[:,-3:]).any()

True

In [ ]:
extracted_features_collapsed_a_e = np.concatenate((extracted_features_collapsed_a[:,:-9],extracted_features_collapsed_a[:,-6:-3]),axis=1)

In [ ]:
extracted_features_collapsed_b_e = np.concatenate((extracted_features_collapsed_b[:,:-9],extracted_features_collapsed_b[:,-6:-3]), axis=1)

In [ ]:
extracted_features_collapsed_e = np.concatenate((extracted_features_collapsed[:,:-9],extracted_features_collapsed[:,-6:-3]), axis=1)

In [75]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
svc = make_pipeline(StandardScaler(), SVC(gamma='auto', C=6.5))
svc.fit(extracted_features_collapsed_a[:,:-1], labels_collapsed_a)
np.sum(labels_collapsed_b == svc.predict(extracted_features_collapsed_b[:,:-1]))/len(labels_collapsed_b)

0.7549234135667396

In [76]:
from sklearn.ensemble import BaggingClassifier
bag = make_pipeline(StandardScaler(), BaggingClassifier(n_estimators=1000, random_state=0))
bag.fit(extracted_features_collapsed_a[:,:-1], labels_collapsed_a)
np.sum(labels_collapsed_b == bag.predict(extracted_features_collapsed_b[:,:-1]))/len(labels_collapsed_b)
#0.772

0.7724288840262582

In [ ]:
bag = make_pipeline(StandardScaler(), BaggingClassifier(base_estimator=SVC(gamma='auto', C=6.5), n_estimators=20, random_state=0))
bag.fit(extracted_features_collapsed_e, labels_collapsed)
np.sum(labels_collapsed == bag.predict(extracted_features_collapsed_e))/len(labels_collapsed)

0.8662963697679031

In [ ]:
extracted_features_collapsed_e.shape

(5041, 83)

In [ ]:
np.sum(labels_collapsed_a == svc.predict(extracted_features_collapsed_a_e))/len(labels_collapsed_a)

0.8660087719298246

In [ ]:
gbt = GradientBoostingClassifier(n_estimators=700, learning_rate=0.1, random_state=1)
gbt.fit(extracted_features_collapsed_a, labels_collapsed_a, weighted_labels[:4600])
np.sum(labels_collapsed_b == gbt.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.6340956340956341

In [ ]:
np.sum(labels_collapsed_a == gbt.predict(extracted_features_collapsed_a))/len(labels_collapsed_a)

0.9923913043478261

In [ ]:
np.sum(labels_collapsed_b == gbt.predict(extracted_features_collapsed_b))/len(labels_collapsed_b)

0.6340956340956341

--------------------------------------------------Making predictions-----------------------------------------------

In [112]:
X_test = test_X_tensor.numpy()

In [113]:
X_test.shape

(3411, 17842)

In [114]:
extracted_rpeaks_test=[]

In [115]:
for i in range(len(X_test)):
    signal = X_test[i]
    signal = signal[~np.isnan(signal)] #drop nans
    if (i%500==0):
      print(i)
    res = ecg.ecg(signal, sampling_rate=300, show=False)
    r_peaks = res['rpeaks']
    heart_rates = res["heart_rate"]
    if len(r_peaks) < 5:
      print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # else:
    #   print(f"len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
    # r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks'] #300 is the sampling rate afaik
    extracted_rpeaks_test.append([signal,r_peaks,heart_rates])    

0
500
1000
1500
2000
2500
3000


In [180]:
extracted_features_test = np.ones((len(X_test),5*27+5*4+2+2+2+2*4+1+1+61))

In [ ]:
for i in range(len(extracted_rpeaks_test)):
    signal, r_peaks, heart_rates = (extracted_rpeaks_test[i][0],extracted_rpeaks_test[i][1],extracted_rpeaks_test[i][2])
    if (i%500==0):
      print(f"processing point {i}")
    if len(r_peaks) >= 5:
        beats = ecg.extract_heartbeats(signal,r_peaks,300,before=0.45,after=0.45)['templates']
        if len(beats) != 0:
            r_diffs = np.diff(r_peaks)
            mu = np.mean(beats, axis=0)[0::10] 
            mini = np.amin(beats, axis=0)[0::10] 
            maxa = np.amax(beats, axis=0)[0::10]  
            var = np.std(beats, axis=0)[0::10] 
            med = np.median(beats, axis=0)[0::10] 
            r_mu = np.mean(r_diffs)
            r_mini = np.amin(r_diffs) 
            r_maxa = np.amax(r_diffs)  
            r_var = np.std(r_diffs)
            r_med = np.median(r_diffs)
            smentropy = ant.sample_entropy(mu)
            appentropy = ant.app_entropy(mu)

            r_diffs_diffs = np.diff(r_diffs)
            r_diffs_mu = np.mean(r_diffs_diffs)
            r_diffs_maxa = np.amax(r_diffs_diffs)
            r_diffs_mini = np.amin(r_diffs_diffs)
            r_diffs_var = np.std(r_diffs_diffs)
            r_diffs_med = np.median(r_diffs_diffs)
            r_diffs_25 = np.percentile(r_diffs_diffs,25)
            r_diffs_75 = np.percentile(r_diffs_diffs,75)
            nn_50 = len(r_diffs_diffs[r_diffs_diffs>15]) / len(r_diffs_diffs)
            nn_20 = len(r_diffs_diffs[r_diffs_diffs>6]) / len(r_diffs_diffs)
            c = np.mean(sorted(heart_rates[:-2]))*len(signal)/18000
            rat = np.abs((len(r_peaks)/c)-1)

            r_diffs_diffs_2 = np.diff(r_diffs_diffs)
            r_diffs2_mu = np.mean(r_diffs_diffs_2)
            r_diffs2_maxa = np.amax(r_diffs_diffs_2)
            r_diffs2_mini = np.amin(r_diffs_diffs_2)
            r_diffs2_var = np.std(r_diffs_diffs_2)
            r_diffs2_med = np.median(r_diffs_diffs_2)
            r_diffs2 = np.array([r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med])

            abla, lospowers = scipy.signal.welch(signal, 300, 'flattop', 1024, scaling='spectrum')

            #heart rate stats
            if len(heart_rates)>0:
              hr_mu = np.mean(heart_rates)
              hr_maxa = np.amax(heart_rates)
              hr_mini = np.amin(heart_rates)
              hr_var = np.std(heart_rates)
              hr_med = np.median(heart_rates)
            else:
              hr_mu = np.nan
              hr_maxa = np.nan
              hr_mini = np.nan
              hr_var = np.nan
              hr_med = np.nan
            r_stats = np.array([r_mu,r_var,r_mini,r_maxa,r_med,r_diffs_mu,r_diffs_maxa,r_diffs_mini,r_diffs_var,r_diffs_med, r_diffs_25, r_diffs_75,hr_mu,hr_maxa,hr_mini,hr_var,hr_med,nn_50,nn_20,smentropy,appentropy,rat])
            r_stats = np.concatenate((r_stats,r_diffs2,lospowers[0:10],lospowers[10::10]))

            #HRV STATS

            r_peaks_s = r_peaks/300
            a = welch_psd(rpeaks=r_peaks_s, fbands = fbands, show=False, show_param=False, legend=False)#, fbands=None)
            hrv_stats = np.concatenate((a["fft_peak"],a["fft_abs"],np.array([a["fft_ratio"]])))
            extracted_features_test[i] = np.array(np.concatenate((mu,var,mini,maxa,med,r_stats,hrv_stats)))
    else:  
        print(f"Inappropriate input encountered: len(r_peaks) = {len(r_peaks)}, len(signal)={len(signal)} index is {i}")
        extracted_features_test[i]=np.ones(5*27+5*4+2+2+2+2*4+1+1+61) #np.array([np.ones(18),np.ones(18),np.ones(18),np.ones(18)])

In [ ]:
# extracted_features_test = np.apply_along_axis(extract_features, 1, X_test)

In [182]:
bool_extracted_test = (extracted_features_test == np.array(np.ones(5*27+5*4+2+2+2+2*4+1+1+61))).all(axis=1)

In [183]:
extracted_features_test.shape

(3411, 232)

In [184]:
extracted_features_test_valid = extracted_features_test[~bool_extracted_test]
extracted_features_test_valid.shape

(3411, 232)

In [185]:
predictiontest = np.ones(3411)

In [ ]:
#predictiontest[~bool_extracted_test] = xgb.predict(extracted_features_test_valid)

In [178]:
p1 = xgb.predict(extracted_features_test_valid)

In [186]:
p1 = lgb.predict(extracted_features_test_valid)

In [ ]:
#@title ablaoubla
# from scipy import stats
# comb = np.concatenate((p1[:,np.newaxis],p2[:,np.newaxis],p3[:,np.newaxis]),axis=1)
# comb.shape

(3356, 3)

In [124]:
# def get_mode(a):
#   return stats.mode(a)[0][0]
p1.shape

(3411,)

In [187]:
predictiontest[~bool_extracted_test]  = p1 #np.apply_along_axis(get_mode, 1, comb)

In [67]:
predictiontest[bool_extracted_test] = 0

In [ ]:
list(predictiontest).count(3)

121

In [188]:
test_y = pd.read_csv('sample.csv')
test_y.loc[:,"y"] = predictiontest
test_y.to_csv("predictions16.csv", index=False)